# Рекомендация тарифов

В yашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Построим модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверить *accuracy* на тестовой выборке самостоятельно.

## 1. Откроем и изучим файл

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import random
import joblib
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()
data['is_ultra'].mean()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


0.30647168637212197

### Изучив общую информацию по датасету, мы имеем:
1. Датасет состоит из 3214 пользователей;
2. Всего параметров в датасете 5 - количество звонков, продолжительность звонков, количество смс, израсходованный интернет-трафик, используемый тариф;
3. 70% абонентов используют тариф "смарт", 30% - тариф "ультра"

## 2. Разобьем данные на выборки

Разобъем данные на обучающую(data-train), валидационную(data_valid), тестовую(data_test) в соотношении 60/20/20. Для этого используем train_test_split. В каждой группе выделим набор признаков и целевой признак. Проверим в разбитых группах соотношение тарифов "смарт" и "ультра".

In [2]:
data_train, data_testing = train_test_split(data, test_size=0.4, random_state=12345)
data_valid, data_test = train_test_split(data_testing, test_size=0.5, random_state=12345)
print(data_train.shape)
print(data_valid.shape)
print(data_test.shape)
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']
features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']
features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']
print(data_train['is_ultra'].mean())
print(data_valid['is_ultra'].mean())
print(data_test['is_ultra'].mean())
    

(1928, 5)
(643, 5)
(643, 5)
0.30757261410788383
0.2939346811819596
0.3157076205287714


### Вывод
1. В обучающую, валидационную и тестовую группы попали 1928, 643, 643 абонента соответственно.
2. Соотношение тарифов в каждой группе осталось неизменным - примерно 70/30.


## 3. Исследуем модели

3.1 Модель логической регрессии

In [3]:
model_1 = LogisticRegression(random_state=12345, solver='liblinear')
model_1.fit(features_train, target_train)
predictions_1_train = model_1.predict(features_train)
predictions_1_valid = model_1.predict(features_valid)
print("Accuracy")
print("Обучающая выборка:", accuracy_score(target_train, predictions_1_train))
print("Валидационная выборка:", accuracy_score(target_valid, predictions_1_valid))

Accuracy
Обучающая выборка: 0.7505186721991701
Валидационная выборка: 0.7589424572317263


In [ ]:
get_dummies()

3.2 Модель решающего дерева

In [4]:
best_tree = []
for i in range(1, 20):
    for j in range(2, 20, 2):
        for k in range(1, 10):
            model_2 = DecisionTreeClassifier(random_state=12345, max_depth=i, min_samples_split=j, min_samples_leaf=k)
            model_2.fit(features_train, target_train)
            predictions_2_train = model_2.predict(features_train)
            predictions_2_valid = model_2.predict(features_valid)
            accuracy_train = accuracy_score(target_train, predictions_2_train)
            accuracy_valid = accuracy_score(target_valid, predictions_2_valid)
            best_tree.append([i, j, k, accuracy_train, accuracy_valid])                               
best_tree = pd.DataFrame(data = best_tree,\
                         columns = ['max_depth', 'split', 'leaf', 'accuracy_train', 'accuracy_valid'])\
                         .sort_values(by = 'accuracy_valid', ascending=False)
best_tree

,max_depth,split,leaf,accuracy_train,accuracy_valid
478,6,18,2,0.832469,0.790047
480,6,18,4,0.831432,0.788491
479,6,18,3,0.832469,0.788491
461,6,14,3,0.832469,0.786936
408,6,2,4,0.834025,0.786936
...,...,...,...,...,...
1309,17,4,5,0.892635,0.723173
1462,19,2,5,0.893672,0.723173
1480,19,6,5,0.893672,0.723173
1471,19,4,5,0.893672,0.723173


3.3 Модель случайного леса

In [5]:
best_forest = []
for estim in range(5, 50, 5):
    for i in range(1, 10):
        for j in range(2, 20, 2):
            for k in range(1, 6):
                model_3 = RandomForestClassifier(n_estimators=estim,\
                                                 max_depth=i,\
                                                 random_state=12345,\
                                                 min_samples_split=j, 
                                                 min_samples_leaf=k)
                model_3.fit(features_train, target_train)
                predictions_3_train = model_3.predict(features_train)
                predictions_3_valid = model_3.predict(features_valid)
                accuracy_train = accuracy_score(target_train, predictions_3_train)
                accuracy_valid = accuracy_score(target_valid, predictions_3_valid)
                best_forest.append([estim, i, j, k, accuracy_train, accuracy_valid])
best_forest = pd.DataFrame(data = best_forest,\
                         columns = ['n_estimators', 'max_depth', 'split', 'leaf', 'accuracy_train', 'accuracy_valid'])\
                         .sort_values(by = 'accuracy_valid', ascending=False)
best_forest


,n_estimators,max_depth,split,leaf,accuracy_train,accuracy_valid
2377,30,8,16,3,0.849585,0.811820
2782,35,8,16,3,0.848548,0.811820
3110,40,7,4,1,0.857365,0.810264
3628,45,9,12,4,0.857365,0.808709
2661,35,6,4,2,0.835062,0.808709
...,...,...,...,...,...,...
839,15,1,12,5,0.744295,0.752722
838,15,1,12,4,0.744295,0.752722
837,15,1,12,3,0.744295,0.752722
836,15,1,12,2,0.744295,0.752722


### Вывод
Результаты тестирования 3-х моделей показали:
1. Точность логистической регрессии - 0.7589
2. Лучшая точность решающего дерева - 0.79
3. Лучшая точность случайного леса - 0.8118.
### За лучшую модель примем случайный лес.

In [6]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']
model_5 = LogisticRegression(random_state=12345, solver='liblinear')
scores = cross_val_score(model_5, features, target, cv=5)
scores
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.71 (+/- 0.03)


## 4. Проверьте модель на тестовой выборке

In [7]:
model_4 = RandomForestClassifier(n_estimators=30,\
                                 max_depth=8,\
                                 random_state=12345,\
                                 min_samples_split = 16,\
                                 min_samples_leaf = 3)
model_4.fit(features_train, target_train)
predictions_4_test = model_4.predict(features_test)
print (accuracy_score(target_test, predictions_4_test))
model_4.feature_importances_

0.8040435458786936


array([0.19287866, 0.20800558, 0.22413893, 0.37497683])

In [8]:
scores_1 = cross_val_score(model_1, features, target, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_1.mean(), scores_1.std() * 2))
scores_2 = cross_val_score(model_2, features, target, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_2.mean(), scores_2.std() * 2))
scores_3 = cross_val_score(model_3, features, target, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores_3.mean(), scores_3.std() * 2))

Accuracy: 0.71 (+/- 0.03)
Accuracy: 0.77 (+/- 0.02)
Accuracy: 0.81 (+/- 0.02)


### Вывод
Выбранная модель случайного леса на тестовой выборке показала точность - 0.804, что удовлетворяет условиям задачи - довести долю правильных ответов по крайней мере до 0.75. Исследуя значимость параметров для данной моделей, можно сделать вывод, что наибольшую значимость в выборе тарифа имеет израсходованный интернет-трафик.

## 5. (бонус) Проверим модели на адекватность

Сравним выбранную модель со случайной. Для этого создадим рандомные результаты тестовой выборке в переменной random_predictions и сравним точность данных результатов с точностью расчетной модели.

In [9]:
random_predictions = [random.randint(0, 1) for _ in range(len(data_test.index))]
if accuracy_score(target_test, random_predictions) < accuracy_score(target_test, predictions_4_test):
    print('Показатель accuracy расчетной модели:', accuracy_score(target_test, predictions_4_test))
    print('Показатель accuracy случайной модели:', accuracy_score(target_test, random_predictions))
    print('Точность расчетной модели лучше случайной')

Показатель accuracy расчетной модели: 0.8040435458786936
Показатель accuracy случайной модели: 0.5163297045101088
Точность расчетной модели лучше случайной


## Вывод
1. Рассмотрено 3 модели: логическая регрессия, решающий лес, случайный лес.
2. Лучшие результаты показала модель сдучайного леса.
3. С помощью выбранной модели достигнут accuracy не меньше 0.75 - 0.804.
4. Выбранная модель проверена на адекватность, с помощью сравнения со случайной моделью.